# South Africa Population Density Visualization

Json shapefile: https://github.com/deldersveld/topojson/tree/master/countries/south-africa
</p>Shape file mapping: 0:EC, 1:FS, 2:GT, 3:KZN, 4:LIM, 5:MP, 6:NW, 7:NC, 8:WC

Population Data: https://www.southafricanmi.com/population-density-map.html
</p>Date of Article Update: 30 July 2019

Date of visualization: March 2020

In [1]:
import pandas as pd

# packages for retrieving coordinates
from selenium import webdriver
from time import sleep
import urllib

# packages for map plots
import folium
import json
import branca
import html 

pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 500)

In [2]:
# population data: 
pop_data = pd.read_excel('Population Data.xlsx')

In [3]:
# mapping comes from json file
data_key = pd.DataFrame(data = {'Key': [0,1,2,3,4,5,6,7,8], 
                                'Province':['Eastern Cape', 'Free State', 'Gauteng', 'KwaZulu-Natal', 'Limpopo', 'Mpumalanga', 'North West', 'Northern Cape', 'Western Cape']})

In [4]:
pop_data = pop_data.merge(data_key, on='Province')

In [5]:
# retrieve coordincates of capital cities
driver = webdriver.Chrome()

def build_url(address):
    quoted_address = urllib.parse.quote_plus(address)
    prefix = "https://www.google.com/maps/search/?api=1&query="
    return  prefix + quoted_address

def retrieve_geocode(url):
    url = build_url(url)
    driver.get(url)
    sleep(6)
    return driver.current_url.split("!")[-2:]

geocoded_data = []

for d in pop_data.Capital:
    latlong = retrieve_geocode(d)
    
    #print("{} {}".format(d, latlong))   
    geocoded_data.append((d, latlong))

addresses = [v[0] for v in geocoded_data]
lats = [v[1][0].replace("3d", "") for v in geocoded_data]
longs = [v[1][1].replace("4d", "") for v in geocoded_data]

geocoded_df = pd.DataFrame({'Capital':addresses, 'latitude':lats, 'longitude':longs})

In [6]:
pop_data = pop_data.merge(geocoded_df, on='Capital')
# set Key as index for plotting
pop_data = pop_data.set_index(pop_data.Key)
pop_data

,Province,Capital,Population,SquareKilometers,PopulationDensity,Key,latitude,longitude
Key,,,,,,,,
8,Western Cape,Cape Town,6621103,129462,51.143216,8,-33.9248685,18.4240553
0,Eastern Cape,Bhisho,6522734,168966,38.603826,0,-32.849876,27.4409557
7,Northern Cape,Kimerbley,1225555,372889,3.286648,7,-28.7282384,24.7499112
1,Free State,Bloemfontein,2954348,129825,22.756387,1,-29.085214,26.1595761
3,KwaZulu-Natal,Pietermaritzburg,11384722,94361,120.650714,3,-29.6006068,30.3794118
6,North West,Mahikeng,3978955,104882,37.937444,6,-25.855978,25.64031
2,Gauteng,Johannesburg,14717040,18178,809.607218,2,-26.2041028,28.0473051
5,Mpumalanga,Mbombela,4523874,76495,59.139473,5,-25.4752984,30.9694163
4,Limpopo,Polokwane,5797275,125755,46.099757,4,-23.8961708,29.4486263


The following graph shows:
- Background colour is determined by Population Density
- Pop-ups are where the provinces's capital is located
- Pop-ups are marked red if the province's population density is above 100
- Inside the pop-up frame includes information about the province

In [11]:
m = folium.Map(location=[-29.0852, 26.1596], zoom_start=5)

data = json.load(open('za-provinces.json'))

colour_scale = branca.colormap.linear.YlOrRd_09.scale(0, 150)

def style_function(feature):
    return{
        'fillOpacity' : 0.5,
        'weight' : 0,
        'fillColor' : colour_scale(pop_data.loc[(feature['id']), 'PopulationDensity'])
    }

folium.TopoJson(data, 'objects.layer1', name='topojson', style_function=style_function).add_to(m)

tooltip = 'More Info'

for row in pop_data.iterrows():
    # set display information 
    html="""
        Province: {}<br>
        Capital: {}<br>
        Est. Population: {:,.0f} <br>
        Square Kilometers: {:,.0f} <br>
        Population Density: {:,.2f}
        """.format(row[1]['Province'], 
                   row[1]['Capital'], 
                   row[1]['Population'], 
                   row[1]['SquareKilometers'], 
                   row[1]['PopulationDensity'] )
    
    iframe = folium.IFrame(html=html, width=250, height=120)
    popup = folium.Popup(iframe, max_width=500)
    
    # mark red if population Density exceeds 100
    if row[1]['PopulationDensity'] > 100:
        colour = 'red'
    else:
        colour = 'blue'
        
    # add marker to map
    folium.Marker([float(row[1]['latitude']), float(row[1]['longitude'])], 
                  popup=popup, tooltip=tooltip, icon=folium.Icon(color=colour)).add_to(m)

# write to html
# m.save('PoulationMap.html')
m